<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_tensorflow2/blob/main/6_ChatBot/Transformer_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

In [ ]:
b=1-tf.linalg.band_part(tf.ones((2,2)),-1,0)

In [ ]:
b*-1e9

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.e+00, -1.e+09],
       [-0.e+00, -0.e+00]], dtype=float32)>

In [1]:
import numpy as np

In [ ]:
a=np.array([[1,2],[3,4]])

In [ ]:
a+=b*-1e9

In [ ]:
a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.e+00, -1.e+09],
       [ 3.e+00,  4.e+00]], dtype=float32)>

In [ ]:
a=np.array([[[[1,2,3],[3,4,5]],[[5,6,7],[7,8,9]]],[[[1,2,3],[3,4,5]],[[5,6,7],[7,8,9]]]])

In [ ]:
a.shape

(2, 2, 2, 3)

In [ ]:
tf.matmul(a,a,transpose_b=True)

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int64, numpy=
array([[[[ 14,  26],
         [ 26,  50]],

        [[110, 146],
         [146, 194]]],


       [[[ 14,  26],
         [ 26,  50]],

        [[110, 146],
         [146, 194]]]])>

### tf.reshape (4차원)

In [3]:
a = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
a

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]])

In [8]:
b=tf.reshape(a,(2,-1,3,1))
b

<tf.Tensor: shape=(2, 2, 3, 1), dtype=int64, numpy=
array([[[[ 1],
         [ 2],
         [ 3]],

        [[ 4],
         [ 5],
         [ 6]]],


       [[[ 7],
         [ 8],
         [ 9]],

        [[10],
         [11],
         [12]]]])>

In [9]:
tf.transpose(b,[0,2,1,3])

<tf.Tensor: shape=(2, 3, 2, 1), dtype=int64, numpy=
array([[[[ 1],
         [ 4]],

        [[ 2],
         [ 5]],

        [[ 3],
         [ 6]]],


       [[[ 7],
         [10]],

        [[ 8],
         [11]],

        [[ 9],
         [12]]]])>

In [10]:
tf.reshape(a,(2,3,-1,1))

<tf.Tensor: shape=(2, 3, 2, 1), dtype=int64, numpy=
array([[[[ 1],
         [ 2]],

        [[ 3],
         [ 4]],

        [[ 5],
         [ 6]]],


       [[[ 7],
         [ 8]],

        [[ 9],
         [10]],

        [[11],
         [12]]]])>

In [11]:
x = tf.constant([[[ 1,  2,  3],
                  [ 4,  5,  6]],
                 [[ 7,  8,  9],
                  [10, 11, 12]]])

In [12]:
tf.transpose(x,perm=[1,2,0])

<tf.Tensor: shape=(2, 3, 2), dtype=int32, numpy=
array([[[ 1,  7],
        [ 2,  8],
        [ 3,  9]],

       [[ 4, 10],
        [ 5, 11],
        [ 6, 12]]], dtype=int32)>

In [13]:
tf.transpose(x,perm=[2,1,0])

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[ 1,  7],
        [ 4, 10]],

       [[ 2,  8],
        [ 5, 11]],

       [[ 3,  9],
        [ 6, 12]]], dtype=int32)>